In [1]:
import numpy as np


def functionality(x, p, k, parties=3, kappa=30):
    #n = x.shape[0]
    d = x.size
    x = np.reshape(x.size, 1) # Make sure we're one-dimensional
    added = x #np.sum(x,axis=0) assume already aggregated

    # sample twice from geometric distribution (or negative binomial with second parameter 1), once for each party
    trusted_noise_parties = np.ceil((parties-1)/2)
    for i in range(parties-1):
        noise = np.random.negative_binomial(n=1./trusted_noise_parties,p=p, size=d)
        added = np.add(added, noise)
    # MISSING: if any resulting noisy share exceeds 2^{kappa+n} bits, round down. This would reduce the total error
    
    share1 = np.random.randint(-2**kappa,2**kappa,size=d)
    share2 = added-share1
    for i in range(d):
        if share1[i]+share2[i] != added[i]: # RP: Happens only if overflow?
            share2[i]=-1*share2[i]
    truncated_true = [np.trunc((s1+s2)/k) for (s1,s2) in zip(share1,share2)]
    truncated_shares = [np.trunc((s1)/k)+np.trunc((s2)/k) for (s1,s2) in zip(share1,share2)]
    truncation_error = [s2 - s1 for (s1,s2) in zip(truncated_true,truncated_shares)]

    # truncation_wo_sharing = np.random.binomial(1,0.5)*0.5
    # truncation error depends on number of parties with shares (parties-1), not including the dealer
    # truncation_error = int(np.sum(np.random.binomial((parties-1),0.5))/2)
    truncated = np.trunc(added/k)-truncation_error
    return np.argmax(truncated)
    

In [2]:
# we assume one of the parties is the dealer and the rest compute on the shares
parties = 3

# number of data owners
n = 100
# number of dimensions
d = 8
# parameter for geometric distribution (computed from DP guarantee)
p = 0.4
# divisor for truncating
k = 2
# data sampled uniformly in {0,1}
x = np.around(np.random.rand(n,d))


print(functionality(x,p,parties,k))

128
